## imports, etc.

In [23]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import torchvision
from tqdm.notebook import tqdm
import os
import sys
import pickle as pkl
import numpy as np
from sklearn.semi_supervised import LabelPropagation
from sklearn.neural_network import MLPClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
np.random.seed(0)

In [3]:
X_train = torch.load("data/FashionMNIST_train_latent.pt")
X_test = torch.load("data/FashionMNIST_test_latent.pt")
y_train = torch.load("data/FashionMNIST_train_labels.pt")
y_test = torch.load("data/FashionMNIST_test_labels.pt")

In [4]:
os.chdir("../fashion_ml")

In [5]:
from fashion_vae import FashionVAE
from data_loader import EZ_Dataloader
from experiment import FashionML

## limit data labels

In [6]:
X = np.array(torch.cat((X_train, X_test), 0))
y = np.array(torch.cat((y_train, y_test), 0))

In [7]:
ind = np.random.randint(len(X), size=(50))
y_lim = np.repeat(-1, len(y))
y_lim[ind] = y[ind]

## semi-supervised clustering

after a grid search, determined n=2 optimal (for any larger n, some classes never predicted)

In [17]:
lpm = LabelPropagation(kernel='knn',n_neighbors=2,n_jobs=-1,max_iter=3000)
lpm.fit(X, y_lim)

/home/kmarke/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/semi_supervised/_label_propagation.py:281: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
/home/kmarke/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/semi_supervised/_label_propagation.py:290: ConvergenceWarning: max_iter=3000 was reached without convergence.
  warnings.warn(


LabelPropagation(kernel='knn', max_iter=3000, n_jobs=-1, n_neighbors=2)

In [20]:
lpm.score(X, y)

0.10055714285714286

In [18]:
y_pred = lpm.predict(X)
np.unique(y_pred)

In [ ]:
lpmodel = LabelPropagation(kernel='knn',n_neighbors=100,n_jobs=-1,max_iter=3000)
lpmodel.fit(X, y_lim)

/home/kmarke/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/semi_supervised/_label_propagation.py:281: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer


In [24]:
lpmodel.score(X, y)

## mlp models